# Manage asynchronous application jobs

With the Writer API, you can invoke no-code applications asynchronously and in batches using the [async applications endpoints](https://dev.writer.com/api-guides/async-applications).
    
This notebook shows how to create, retrieve, and retry asynchronous no-code application jobs. 

## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/aistudio/signup?utm_campaign=devrel) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)
- A [no-code application](https://dev.writer.com/no-code/building-a-chat-app) deployed in Writer AI Studio
    - If you don't have an application deployed, follow [this guide](https://dev.writer.com/no-code/building-a-chat-app) to build and deploy a chat application.
    - Store the application ID for later use in this tutorial. You can find the application ID in the URL of the application in the Writer AI Studio UI. The URL structure is: `https://app.writer.com/aistudio/organization/<organization-id>/app/<app-id>`.

## Setup

Install the Writer SDK:

In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()

## Usage

The following code asks you to enter your application ID and then retrieves the application metadata, including the required application inputs. It then asks you to enter the values for the application inputs, so it can later create an async job with those values.

In [ ]:
application_id = input("Enter your application ID: ")
application = client.applications.retrieve(application_id=application_id)
inputs = []
for input_ in application.inputs:
    value = input("Enter the value for input {}: ".format(input_.name))
    inputs.append({"id": input_.name, "value": [value]})

### Create an asynchronous job

The code below takes the inputs you defined above for your application and creates an async job.

In [ ]:
job = client.applications.jobs.create(
    application_id=application_id,
    inputs=inputs
)

print(f"Job ID: {job.id}. Status: {job.status}")

### Retrieve a job by ID

To retrieve a job by its ID, call the `applications.jobs.retrieve` function and pass in the job ID.

You can see if the job has finished by inspecting the `status` field. Once the job is complete, the status will be `completed` and the results will be in the `job.data` field.

In [ ]:
job_by_id = client.applications.jobs.retrieve(
    job_id=job.id,
)

print(job.status)
if job.status == "completed":
    print(job.data.suggestion)

### Retrieve all jobs for an application

You can retrieve all jobs for an application with the `applications.jobs.list` method. The result is an automatically paginated iterator that will yield the next page of results as you iterate.

In [ ]:
page = client.applications.jobs.list(
    application_id=application_id,
)

for job in page.result:
    print(f"{job.id}: {job.status}")

You can customize pagination with the `offset` and `limit` parameters. The code sample below demonstrates offsetting the results by 1 and limiting the results to 1 item per page.

In [ ]:
page_pagination = client.applications.jobs.list(
    application_id=application_id,
    offset=1,
    limit=1,
)

for job in page_pagination.result:
    print(f"{job.id}: {job.status}")

## Retry a failed job

If a job fails, you can retry it by calling the `applications.jobs.retry` method and passing in the failed job ID.

The following code creates a job that will fail.

In [ ]:
failed_job = client.applications.jobs.create(
    application_id=application_id,
    inputs=[{"id" :"wrong_name", "value": ["Write a report on the benefits of using AI in research"]}],
)
print(failed_job.id)

The next code sample retries the failed job with the `retry` method.

In [ ]:

response = client.applications.jobs.retry(
    job_id=failed_job.id,
)

print(response)